In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Energy Data.csv')

In [3]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,1/11/2016 17:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,1/11/2016 17:10,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,1/11/2016 17:20,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,1/11/2016 17:30,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0,45.410390,45.410390
4,1/11/2016 17:40,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [4]:
df.isna().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [5]:
df = df.drop(columns=['date', 'lights'])

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
scaler = MinMaxScaler()
normalized_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [8]:
features_df = normalized_df.drop(columns=['Appliances'])
target_variable = normalized_df['Appliances']

In [9]:
X = normalized_df.iloc[:, 3].values #T2
Y = normalized_df.iloc[:, 11].values #T6

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
regressor = LinearRegression()
X_train = X_train.reshape(-1,1)
Y_train = Y_train.reshape(-1,1)
regressor.fit(X_train, Y_train)

LinearRegression()

In [14]:
X_test = X_test.reshape(-1,1)
y_pred = regressor.predict(X_test)

In [15]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [16]:
R2_SCORE = r2_score(Y_test, y_pred)
round(R2_SCORE,2)

0.64

In [17]:
x_train, x_test, y_train, y_test = train_test_split(features_df, target_variable, test_size=0.30, random_state=42)

In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

LinearRegression()

In [20]:
predicted_values = linear_model.predict(x_test)

In [21]:
MAE = mean_absolute_error(y_test, predicted_values)
round(MAE,2)

0.05

In [22]:
RSS = np.sum(np.square(y_test-predicted_values))
round(RSS, 2)

45.35

In [23]:
RMSE = np.sqrt(mean_squared_error(y_test, predicted_values))
round(RMSE,3)

0.088

In [24]:
R2_SCORE = r2_score(y_test, predicted_values)
round(R2_SCORE,2)

0.15

In [25]:
def get_weights_df(model, feat, col_name):
    weights= pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [26]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_weight')

In [27]:
linear_model_weights

,Features,Linear_Model_weight
0,RH_2,-0.456662
1,T_out,-0.321969
2,T2,-0.236198
3,T9,-0.189977
4,RH_8,-0.157585
5,RH_out,-0.077748
6,RH_7,-0.044620
7,RH_9,-0.039808
8,T5,-0.015645
9,T1,-0.003242


In [28]:
from sklearn.linear_model import Ridge

In [29]:
ridgereg = Ridge(alpha=0.4)
ridgereg.fit(x_train, y_train)

Ridge(alpha=0.4)

In [30]:
pred2 = ridgereg.predict(x_test)

In [31]:
RMSE = np.sqrt(mean_squared_error(y_test, pred2))
round(RMSE,3)

0.088

In [32]:
from sklearn.linear_model import Lasso

In [33]:
LassoReg = Lasso(alpha=0.001)
LassoReg.fit(x_train, y_train)

Lasso(alpha=0.001)

In [34]:
pred3 = LassoReg.predict(x_test)

In [35]:
lasso_weights_df = get_weights_df(LassoReg, x_train, 'Lasso_weight')

In [36]:
lasso_weights_df

,Features,Lasso_weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [37]:
RMSE = np.sqrt(mean_squared_error(y_test, pred3))
round(RMSE,3)

0.094